## CSC14120 – PARALLEL PROGRAMMING - Final project report

### Authors
Student 1: Huynh Minh Tuan - 20120024  
Student 2: Huynh Minh Tu - 2012  

### Problem statement
Implementing and optimizing the forward-pass of convolutional layers in modified LeNet-5 using CUDA.

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!nvidia-smi

Mon Dec 25 14:45:49 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 ...    Off | 00000000:01:00.0  On |                  N/A |
| N/A   45C    P8               2W /  60W |     59MiB /  8188MiB |     20%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Convolutional layer in CNN
The figure below shows the process of a convolutional layer.

<img src="images/conv.png" width="800" align="center"/>

### Convolutional layer optimization using CUDA
#### 1. Architecture
<img src="images/optimize_conv.png" width="1000" align="center"/>

#### 2. Versions
In the codebase, we have 3 versions for the optimization
- Version 1: Simple conv implementation, ultilizing parallel processing in cuda.
- Version 2: Using tiled shared memory convolution.
- Version 3: Using cuda streams to handle a batch data sample simultaneously.

The figure above illustrates the workflow of version 3, which is the optimal and fastest version.

### Results
- To make sure that our conv implementation returns correct value, we have trained model with the given data MNIST Fashion. The accuracy on the test data is around 0.82.
- For benchmarking, we evaluate each version of cuda conv on the test dataset. Note that we only need to focus the elapsed time of layer 1 and 4, which are convolutional layers.

In [21]:
### Version 0 (host version)
!bazel run --noshow_progress //:inference --config=report

(15:23:13) INFO: Current date is 2023-12-25
(15:23:13) INFO: Build option --cxxopt has changed, discarding analysis cache.
(15:23:13) INFO: Analyzed target //:inference (0 packages loaded, 2367 targets configured).
(15:23:13) INFO: Found 1 target...
Target //:inference up-to-date:
  bazel-bin/inference
(15:23:14) INFO: Elapsed time: 1.813s, Critical Path: 1.70s
(15:23:14) INFO: 15 processes: 1 internal, 14 local.
(15:23:14) INFO: Running command line: bazel-bin/inference
Object loaded from binary file: weights/lenet5_mnist_weight
--------------------------------
|  Network | Elapsed Time (ms) |
--------------------------------
| Layer 1  |             13690 |
| Layer 2  |               178 |
| Layer 3  |              4288 |
| Layer 4  |             10399 |
| Layer 5  |                52 |
| Layer 6  |              1308 |
| Layer 7  |               613 |
| Layer 8  |                 5 |
| Layer 9  |               216 |
| Layer 10 |                 3 |
| Layer 11 |                21 |
| 

In [18]:
### Version 1
!bazel run --noshow_progress //:inference --config=cuda --config=report --//:conv_ver=v1

(15:22:32) INFO: Current date is 2023-12-25
(15:22:32) INFO: Analyzed target //:inference (0 packages loaded, 0 targets configured).
(15:22:32) INFO: Found 1 target...
Target //:inference up-to-date:
  bazel-bin/inference
(15:22:32) INFO: Elapsed time: 0.053s, Critical Path: 0.00s
(15:22:32) INFO: 1 process: 1 internal.
(15:22:32) INFO: Running command line: bazel-bin/inference
Object loaded from binary file: weights/lenet5_mnist_weight
--------------------------------
|  Network | Elapsed Time (ms) |
--------------------------------
| Layer 1  |               519 |
| Layer 2  |               182 |
| Layer 3  |              4324 |
| Layer 4  |              1398 |
| Layer 5  |                54 |
| Layer 6  |              1315 |
| Layer 7  |               611 |
| Layer 8  |                 6 |
| Layer 9  |               216 |
| Layer 10 |                 3 |
| Layer 11 |                21 |
| Layer 12 |                10 |
--------------------------------
Test acc = 0.8297


In [19]:
### Version 2
!bazel run --noshow_progress //:inference --config=cuda  --config=report --//:conv_ver=v2

(15:22:44) INFO: Current date is 2023-12-25
(15:22:44) INFO: Build option --//:conv_ver has changed, discarding analysis cache.
(15:22:44) INFO: Analyzed target //:inference (0 packages loaded, 2367 targets configured).
(15:22:44) INFO: Found 1 target...
Target //:inference up-to-date:
  bazel-bin/inference
(15:22:46) INFO: Elapsed time: 1.837s, Critical Path: 1.71s
(15:22:46) INFO: 11 processes: 1 internal, 10 local.
(15:22:46) INFO: Running command line: bazel-bin/inference
Object loaded from binary file: weights/lenet5_mnist_weight
--------------------------------
|  Network | Elapsed Time (ms) |
--------------------------------
| Layer 1  |               396 |
| Layer 2  |               183 |
| Layer 3  |              4327 |
| Layer 4  |              1072 |
| Layer 5  |                54 |
| Layer 6  |              1313 |
| Layer 7  |               612 |
| Layer 8  |                 6 |
| Layer 9  |               215 |
| Layer 10 |                 3 |
| Layer 11 |                22

In [20]:
### Version 3
!bazel run --noshow_progress //:inference --config=cuda --config=report --//:conv_ver=v3

(15:22:57) INFO: Current date is 2023-12-25
(15:22:57) INFO: Build option --//:conv_ver has changed, discarding analysis cache.
(15:22:57) INFO: Analyzed target //:inference (0 packages loaded, 2367 targets configured).
(15:22:57) INFO: Found 1 target...
Target //:inference up-to-date:
  bazel-bin/inference
(15:22:59) INFO: Elapsed time: 1.746s, Critical Path: 1.62s
(15:22:59) INFO: 11 processes: 1 internal, 10 local.
(15:22:59) INFO: Running command line: bazel-bin/inference
Object loaded from binary file: weights/lenet5_mnist_weight
--------------------------------
|  Network | Elapsed Time (ms) |
--------------------------------
| Layer 1  |                56 |
| Layer 2  |               179 |
| Layer 3  |              4286 |
| Layer 4  |                28 |
| Layer 5  |                52 |
| Layer 6  |              1305 |
| Layer 7  |               613 |
| Layer 8  |                 6 |
| Layer 9  |               216 |
| Layer 10 |                 3 |
| Layer 11 |                21

All estimated time was calculated in miliseconds.

| Version       | Layer 1          | Layer 4        |
| ------------- | ---------------- | -------------- |
| 0 (host)      | 13690            | 10399          |
| 1             | 519              | 1398           |
| 2             | 396              | 1072           |
| 3             | 56               | 28             | 